In [4]:
%matplotlib inline
import os
import datacube
import warnings
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray
from rasterio.enums import Resampling
from datacube.utils.cog import write_cog
from deafrica_tools.spatial import xr_rasterize
from skimage.morphology import binary_dilation,disk
from skimage.filters.rank import modal
from odc.algo import xr_reproject
import matplotlib.pyplot as plt

crs='epsg:32736' # WGS84/UTM Zone 36S
dict_map={'Water':1,'Cropland':2,'Trees':3,'Bare Surface':4,'Shrubland/Grassland':5,'Irrigated Cropland':6,'Settlements':7} # class value dictionary
# file paths and attributes
lesotho_shp='Data/Mozambique_boundary.shp' # Lesotho boundary shapefile
lesotho_tiles_shp='Data/Mozambique_tiles.shp' # Lesotho tiles shapefile
river_network_shp='Data/hotosm_moz_waterways_lines_filtered.shp' # OSM river network data
road_network_shp='Data/hotosm_moz_roads_lines_filtered.shp' # OSM road network data
google_building_layer='Data/GoogleBuildingLayer_Mozambique_rasterised.tif' # google bulding layer
classification2021_raster='Results/Land_cover2021_postproc_step2_mosaic.tif' # land cover map of 2021
hand_raster='Data/hand_Mozambique.tif' # Hydrologically adjusted elevations, i.e. height above the nearest drainage (hand)

# import Lesotho boundaries and get bounding box
lesotho_polygon=gpd.read_file(lesotho_shp)
[x_min,y_min,x_max,y_max]=lesotho_polygon.total_bounds
print('Lethoso bbox: ',x_min,y_min,x_max,y_max)
lesotho_tiles=gpd.read_file(lesotho_tiles_shp).to_crs(crs) # get bounding boxes of tiles covering Lesotho
tile_bboxes=lesotho_tiles.bounds

# load land cover maps
landcover2021=rioxarray.open_rasterio(classification2021_raster).astype(np.uint8).squeeze() # import land cover map of 2021
landcover2021=landcover2021.rio.reproject(crs) # reproject land cover map 2021

# load external layers
road_network=gpd.read_file(road_network_shp).to_crs(crs) # import OSM road network data and reproject
road_network=road_network.loc[road_network['surface'].isin(['asphalt', 'paved', 'compacted', 'cobblestone', 
                                                             'concrete', 'metal', 'paving_stones', 
                                                             'paving_stones:30'])] # select road network by attributes
road_network.geometry=road_network.geometry.buffer(10) # buffer the road network by 10m
river_network=gpd.read_file(river_network_shp).to_crs(crs) # import OSM river network data and reproject
river_network=river_network.loc[river_network['waterway'].isin(['canal','river'])] # select river network by attribute
hand=xr.open_dataset(hand_raster,engine="rasterio").squeeze() # import hand layer
google_buildings=xr.open_dataset(google_building_layer,engine="rasterio").squeeze() # read in google bulding mask layer

Lethoso bbox:  27.011232336601374 -30.67784748254426 29.4573649650311 -28.57059736718119


In [5]:
for i in range(len(tile_bboxes)):
# for i in range(0,3):
    x_min,y_min,x_max,y_max=tile_bboxes.iloc[i]
    print('Processing tile ',i,'with bbox of ',x_min,y_min,x_max,y_max)
    # clip land cover maps to tile boundary
    landcover2021_tile=landcover2021.rio.clip_box(minx=x_min,miny=y_min,maxx=x_max,maxy=y_max)
    ds_geobox=landcover2021_tile.geobox
    np_landcover2021=landcover2021_tile.squeeze().to_numpy()
    np_landcover2021_post=np_landcover2021.copy()
    # mode filtering for a smoother classification map
    np_landcover2021_post=modal(np_landcover2021,selem=disk(2.5),mask=np_landcover2021!=0)
    # load hand layer
    hand=xr_reproject(hand, ds_geobox, resampling="average")
    np_hand=hand.to_array().squeeze().to_numpy()
    # Make sure water is (only occuring at bottom of watersheds) or fallen within OSM river networks
    river_network_mask=xr_rasterize(gdf=river_network,
                                      da=landcover2021_tile.squeeze(),
                                      transform=ds_geobox.transform,
                                      crs=crs)
    np_river_network_mask=river_network_mask.to_numpy()
    np_landcover2021_post[((np_landcover2021==dict_map['Water'])&(np_hand<=45))|(np_river_network_mask==1)]=dict_map['Water']
    # assign pixels overlapping google building polygons as built-up
    google_buildings_mask=xr_reproject(google_buildings, ds_geobox, resampling="nearest")
    np_google_buildings_mask=google_buildings_mask.to_array().squeeze().to_numpy()
    np_landcover2021_post[np_google_buildings_mask==1]=dict_map['Settlements']
#     # reclassify wetlands around (within 50m of) built-up areas as croplands
#     urban_buffered=binary_dilation(np_landcover2021==dict_map['Settlements'],selem=disk(5))
#     np_landcover2021_post[(urban_buffered==1)&(np_landcover2021==dict_map['Vegetated Wetland'])]=dict_map['Cropland']
    # assign pixesl overlapping OSM road network as built-up class
    road_network_mask=xr_rasterize(gdf=road_network,
                                  da=landcover2021_tile.squeeze(),
                                  transform=ds_geobox.transform,
                                  crs=crs)
    np_road_network_mask=road_network_mask.to_numpy()
    np_landcover2021_post[np_road_network_mask==1]=dict_map['Settlements']
    # convert back result back to DataArray
    landcover2021_tile_post=xr.DataArray(data=np_landcover2021_post,dims=['y','x'],coords={'y':landcover2021_tile.y.to_numpy(), 'x':landcover2021_tile.x.to_numpy()})
    landcover2021_tile_post.rio.write_crs(crs, inplace=True)
    # export as geotiff
    write_cog(landcover2021_tile_post, 'Results/Land_cover2021_postproc_step3_tile_'+str(i)+'.tif', overwrite=True)

Processing tile  0 with bbox of  501087.1133 6738503.8032 551087.1133000001 6788503.8032
Processing tile  1 with bbox of  501087.1133 6688533.8032 551087.1133000001 6738533.8032
Processing tile  2 with bbox of  501087.1133 6638563.8032 551087.1133000001 6688563.8032


In [6]:
! gdal_merge.py -o Results/Land_cover2021_postproc_step3_mosaic.tif -co COMPRESS=Deflate -ot Byte Results/Land_cover2021_postproc_step3_tile_*.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
